In [1]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [4]:
df = pd.read_csv('VAERS/2024VAERSDATA.csv',encoding='windows-1252')
df.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,2728969,01/01/2024,AK,14.0,NaN,NaN,M,NaN,fluzone qiv hd administered to a minor patient...,NaN,...,NaN,NaN,NaN,USSA2023SA396019,2,12/30/2023,NaN,NaN,NaN,NaN
1,2728982,01/01/2024,NJ,NaN,NaN,NaN,U,NaN,administering vaccines subcutaneously to patie...,NaN,...,NaN,NaN,NaN,USGSKUS2023AMR179885,2,01/01/2024,NaN,NaN,NaN,NaN
2,2728984,01/01/2024,NaN,NaN,NaN,NaN,U,NaN,extreme joint pain; Unable to walk; This non-s...,NaN,...,NaN,NaN,NaN,USGSKUS2023AMR180978,2,01/01/2024,NaN,NaN,NaN,NaN
3,2728992,01/01/2024,MO,NaN,NaN,NaN,F,NaN,was due to have 2nd when everything shut down ...,NaN,...,NaN,NaN,NaN,USGSKUS2023AMR181463,2,01/01/2024,NaN,NaN,NaN,NaN
4,2728993,01/01/2024,NaN,NaN,NaN,NaN,U,NaN,Injection site hot; Hypersentivity reaction; C...,NaN,...,Autoimmune disorder (Autoimmune disorder diagn...,NaN,NaN,USGSKUS2023AMR181703,2,01/01/2024,NaN,NaN,NaN,NaN


In [5]:
loader = DataFrameLoader(df,page_content_column="SYMPTOM_TEXT")

data = loader.load()

In [6]:
data[0].page_content

"fluzone qiv hd administered to a minor patient with no reported adverse event; patient was supposed to receive the Flumist Nasal Spray but she grabbed the FLUZONE QIV HD and inadvertently gave it to intranasally with no reported adverse event; patient was supposed to receive the Flumist Nasal Spray but she grabbed the FLUZONE QIV HD and inadvertently gave it to intranasally with no reported adverse event; Initial information received from Regulatory Authority on 18-Dec-2023 regarding an unsolicited valid  non-serious case received from a nurse.  This case involves a 14 years old male patient to whom influenza quadrival A-B high dose HV vaccine [Fluzone High-Dose Quadrivalent] was administered who was supposed to receive the Influenza Vaccine Live Reassort 3v (Flumist) nasal Spary but she grabbed the Fluzone QIV HD and inadvertently gave it to intranasally with no reported adverse event.  The patient's past medical history, medical treatment(s), vaccination(s) and family history were n

In [7]:
from typing import Optional
class events(BaseModel):
    """Information about a patient narrative."""
    VaccineName: str = Field(description="Name of Administered Vaccine if the name of vaccine is not available the value should ba NA")
    HodpitalAdmission: bool = Field(description="This flag is True if patient was admitted into hospital due to Adverse Event otherwise false")
    Age: int = Field(description="Age of the patien if available otherwise the value is NA")

In [8]:
class Information(BaseModel):
    """Information to extract."""
    people: List[events] = Field(description="List of info about patient narratives")

In [9]:
model = ChatOpenAI(temperature=0)


In [10]:
convert_pydantic_to_openai_function(Information)
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess rather provide NA value for non boolean fields. Extract partial info"),
    ("human", "{input}")
])
print(prompt)

input_variables=['input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Extract the relevant information, if not explicitly provided do not guess rather provide NA value for non boolean fields. Extract partial info')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


In [12]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [14]:
vaers=extraction_chain.batch(data[0:20])

In [15]:
vaers

[[{'VaccineName': 'Fluzone High-Dose Quadrivalent',
   'HodpitalAdmission': False,
   'Age': 14}],
 [{'VaccineName': 'NA', 'HodpitalAdmission': False, 'Age': 'NA'}],
 [{'VaccineName': 'RSVPreF3 adjuvanted (Arexvy)',
   'HodpitalAdmission': False,
   'Age': None}],
 [{'VaccineName': 'Herpes zoster (Shingrix)',
   'HodpitalAdmission': False,
   'Age': 54}],
 [{'VaccineName': 'RSVPreF3 adjuvanted (Arexvy)',
   'HodpitalAdmission': False,
   'Age': 25}],
 [{'VaccineName': 'RSVPreF3 adjuvanted (Arexvy)',
   'HodpitalAdmission': False,
   'Age': None}],
 [{'VaccineName': 'Infanrix', 'HodpitalAdmission': False, 'Age': 11}],
 [{'VaccineName': 'Flu Seasonal QIV Quebec (FluLaval Quadrivalent 2023-2024 season)',
   'HodpitalAdmission': False,
   'Age': 75}],
 [{'VaccineName': 'Boostrix', 'HodpitalAdmission': False, 'Age': None}],
 [{'VaccineName': 'Arexvy', 'HodpitalAdmission': False, 'Age': 70}],
 [{'VaccineName': 'Herpes zoster (Shingrix)',
   'HodpitalAdmission': False,
   'Age': None}],
 [{'V

In [16]:
import functools
import operator
#this is reportedly the most efficient way to get one list out of list of lists by concat
vaers_list=functools.reduce(operator.iconcat, vaers, [])


In [18]:
vaers_df = pd.DataFrame.from_dict(vaers_list) 